# Продвинутый SQL

## Задание 1

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

* SELECT COUNT(id)
* FROM stackoverflow.posts
* WHERE post_type_id = 1
AND (score > 300
OR favorites_count >=100)

## Задание 2

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

* SELECT ROUND(AVG(a.cnt_id))
* FROM

         (SELECT COUNT(id) AS cnt_id, DATE_TRUNC('day', creation_date::date) AS dt
         FROM stackoverflow.posts
         WHERE (DATE_TRUNC('day', creation_date::date) BETWEEN '2008-11-01' AND '2008-11-18') AND post_type_id = 1
         GROUP BY dt) AS a;

## Задание 3

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

* select count(distinct b.user_id) as sum
* from stackoverflow.badges as b
* join stackoverflow.users as u on b.user_id=u.id
* where u.creation_date::date=b.creation_date::date


## Задание 4

Сколько уникальных постов пользователя с именем _Joel Coehoorn_ получили хотя бы один голос?

* select count(distinct a.id)
* from stackoverflow.posts as a
* join stackoverflow.users as u on a.user_id=u.id
* JOIN stackoverflow.votes v ON a.id = v.post_id
* WHERE u.display_name = 'Joel Coehoorn'

## Задание 5

Выгрузите все поля таблицы _vote_types_. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю _id_.

* SELECT *,
* ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
* FROM stackoverflow.vote_types
* ORDER BY id

## Задание 6

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

* select u.id, count(vt.name)
* from stackoverflow.users as u
* join stackoverflow.votes as v on u.id=v.user_id
* join stackoverflow.vote_types as vt on v.vote_type_id=vt.id
* WHERE vt.name = 'Close'
* group by u.id
* ORDER BY count(vt.name) DESC, u.id DESC
* LIMIT 10

## Задание 7

* Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
* Отобразите несколько полей:
       
       * идентификатор пользователя;
       * число значков;
       * место в рейтинге — чем больше значков, тем выше рейтинг.
       
* Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
* Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

* select user_id, count(id) c_bad, DENSE_RANK() over(order by count(id) DESC)
* from stackoverflow.badges
* where creation_date::date between '2008-11-15' and '2008-12-15'
* group by user_id
* order by c_bad desc, user_id
* limit 10

## Задание 8

* Сколько в среднем очков получает пост каждого пользователя?

* Сформируйте таблицу из следующих полей:
        
      * заголовок поста;
      * идентификатор пользователя;
      * число очков поста;
      * среднее число очков пользователя за пост, округлённое до целого числа.
      * Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

* WITH ss AS

      (select p.title, p.user_id, p.score, round(avg(score) over (partition by user_id)) as avg_score
       from stackoverflow.posts p
       where p.title != 'NULL' and p.score!=0) 

* SELECT title, user_id, score, avg_score
* FROM ss

## Задание 9

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

* with b as 
        
      (with a as
    (select *,count(user_id) over (partition by user_id) as count_badges
       from stackoverflow.badges)
           select distinct title
           from a
           join stackoverflow.posts as p on p.user_id=a.user_id
           where count_badges>1000)
           select *
           from b
           where title!=''

## Задание 10

* Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). 
* Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

        * пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
        * пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
        * пользователям с числом просмотров меньше 100 — группу 3.
        * Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

* SELECT id, views,

       CASE
           WHEN views >= 350  THEN 1
           WHEN views <350  AND views >= 100 THEN 2
           WHEN views < 100 and views >0 THEN 3
       END
* FROM stackoverflow.users
* WHERE location LIKE '%United States%' and views >0

## Задание 11

* SELECT id, group_number, views
* FROM 

      (SELECT *,MAX(views) OVER (PARTITION BY group_number) AS max_value 
      FROM 
          (SELECT id, views,
                      CASE
                            WHEN views >= 350 THEN 1
                            WHEN views >= 100 THEN 2
                            ELSE 3
                            END AS group_number
           FROM stackoverflow.users AS u
           WHERE location LIKE '%United States%' AND views> 0) AS a) AS foo
* WHERE max_value = views
* ORDER BY views DESC, id

## Задание 12

* Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

        * номер дня;
        * число пользователей, зарегистрированных в этот день;
        * сумму пользователей с накоплением.

* WITH users_1 AS 

      (SELECT EXTRACT(DAY FROM creation_date) AS day_reg, COUNT(id) AS count_id
       FROM stackoverflow.users u
       WHERE EXTRACT(MONTH FROM creation_date) = 11 AND EXTRACT(YEAR FROM creation_date) = 2008
       GROUP BY day_reg)
       
* SELECT *,
* SUM(count_id) OVER (ORDER BY day_reg)
* FROM users_1;


## Задание 13

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. 
Отобразите:

    * идентификатор пользователя;
    * разницу во времени между регистрацией и первым постом.

* SELECT u.id, p.first_post_dt - u.creation_date AS diff
* FROM

      (SELECT DISTINCT user_id,
       FIRST_VALUE(creation_date) OVER (PARTITION BY user_id ORDER BY creation_date) first_post_dt
       FROM stackoverflow.posts) AS p
       
* JOIN stackoverflow.users AS u ON u.id = p.user_id;

## Задание 14

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

* select sum(views_count) as total_views, date_trunc('month', creation_date)::date as month
* from stackoverflow.posts
* group by month
* order by total_views desc

## Задание 15

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

* select distinct u.display_name, count(distinct p.user_id)
* from stackoverflow.users u
* join stackoverflow.posts p on p.user_id=u.id
* join stackoverflow.post_types pt on pt.id=p.post_type_id
* where pt.type = 'Answer' and date_trunc('day', p.creation_date) <= date_trunc('day', u.creation_date) + INTERVAL '1 month'
* group by 1
* having count(p.user_id) > 100
* order by 1

## Задание 16

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

* SELECT DATE_TRUNC('month', p.creation_date) :: date AS month, COUNT(p.id)
* FROM stackoverflow.posts p
* WHERE p.user_id IN

      (SELECT p.user_id AS user_id
       FROM stackoverflow.posts p
      JOIN stackoverflow.users u ON u.id = p.user_id
      WHERE u.creation_date :: date BETWEEN '2008-09-01' AND '2008-09-30'
          AND p.creation_date :: date BETWEEN '2008-12-01' AND '2008-12-31')
          
* GROUP BY month
* ORDER BY month DESC

## Задание 17

* Используя данные о постах, выведите несколько полей:
    
      1) идентификатор пользователя, который написал пост;
      2) дата создания поста;
      3) количество просмотров у текущего поста;
      4) сумму просмотров постов автора с накоплением.
      5) Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

* SELECT p.user_id, p.creation_date, p.views_count, SUM(views_count) OVER (PARTITION BY user_id
ORDER BY creation_date)
* FROM stackoverflow.posts p

## Задание 18

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

* with i as

      (SELECT user_id, COUNT( distinct EXTRACT(DAY from creation_date::date)) AS activ_days
       FROM stackoverflow.posts
       WHERE (creation_date::date BETWEEN '2008-12-01' AND '2008-12-07')
       GROUP BY user_id)

* SELECT round(AVG(i.activ_days))
* FROM i

## Задание 19

* На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:

      * номер месяца;
      * количество постов за месяц;
      * процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
      
* Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.


* with i as

      (select extract(month from creation_date) as month, count(distinct id) as id
       from stackoverflow.posts
       where creation_date::date between '2008-09-01' and '2008-12-31'
       group by 1)

* select month,id, round((id::numeric / lag(id, 1, null) over (order by month))*100-100, 2)
* from i

## Задание 20

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

* WITH i AS

      (SELECT creation_date AS date, EXTRACT (week FROM creation_date) AS week, user_id
       FROM stackoverflow.posts
       WHERE user_id IN 
           (SELECT user_id
            FROM stackoverflow.posts
            GROUP BY user_id
            ORDER BY COUNT(*) DESC
            LIMIT 1))

* SELECT DISTINCT(week), MAX(date) AS date
* FROM i
* WHERE  date::date BETWEEN '2008-10-01' AND '2008-10-31'
* GROUP BY week;